Problem 1
The file linked below contains a data set of 150 samples. The first column contains a single continuous
feature, X, assumed to have been drawn from an unknown probability density. The second column contains
the binary class label Y .

https://f000.backblazeb2.com/file/jeldridge-data/011-univariate_density_estimation/data.csv

In this problem, use a histogram estimator with bins [0, 1.5), [1.5, 3), . . . , [13.5, 15) to estimate the requested
probabilities. In each part, show your code and provide your reasoning.

a) Estimate P(Y = 1 | X = 6.271) directly